In [24]:
%matplotlib inline

import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib import rnn
from sklearn.preprocessing import MinMaxScaler

In [210]:
path = '/Users/kia/Desktop/SYS 6016/HW/hw4/'
file = 'train.txt'
corpus = pd.read_csv(path+file, sep=r'\s{2,}',header=None)
corpus.shape

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


(72193, 1)

In [212]:
train_corpus = corpus.head(100)
train_corpus=np.expand_dims(train_corpus.values,1)
train_corpus = train_corpus.T.tolist()

In [213]:
#convert corpus to list
reviews = list(train_corpus)
reviews = [str(review) for review in reviews]


In [214]:
#split into sentences
lines = [re.split('[?.!]', review.lower()) for review in reviews]
#strip whitespace
lines = [[sentence.strip(' ') for sentence in sentences] for sentences in lines]
#combine sentences from different documents
sentences = [j for i in lines for j in i]
#split sentences into words
words = [re.split(' ', sentence) for sentence in sentences]
words = str(words).split()
words = [''.join(e for e in string if e.isalnum()) for string in words]
words = [''.join(e for e in string if not e.isdigit()) for string in words]
# remove empty strings
words = list(filter(None, words)) 
len(words)

20519

In [215]:
sequences = list()
for i in range(1, len(words)):
    sequence = words[i-1:i+1]
    sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
sequences[0]

Total Sequences: 20518


['we', 'brought']

In [216]:
unique_words = list(set(words))
len(unique_words)

4408

In [217]:
vocab = {}
for i, unique_word in enumerate(unique_words):
  vocab[unique_word] = i

In [218]:
sequence_encodings = []
for sequence in sequences:
    sequence_encoding = [vocab[sequence[0]],vocab[sequence[1]]]
    sequence_encodings.append(sequence_encoding)
len(sequence_encodings)

20518

In [195]:
# prepare data to feed to model:
# each observation will be of length 'seq_size'
# For example, seq_size=2 then
# x[0]=[t_0,t_1]
# x[1]=[t_1,t_2]
# ...
# As for Y, it will also be of length 'seq_size'. why? (hint, sequence-to-sequence)
# For example, seq_size=2 then
# y[0]=[t_1,t_2]
# y[1]=[t_2,t_3]
# ...
seq_size = 1 
trainX, trainY = [], []
for i in range(len(sequence_encodings) - seq_size):
    trainX.append(np.expand_dims(sequence_encodings[i:i+seq_size], axis=1).tolist())
    trainY.append(sequence_encodings[i+1:i+seq_size+1])



In [222]:
trainX, trainY = [], []
seq_size=2
for i in range(1,len(sequence_encodings)):
    trainX.append(np.expand_dims(sequence_encodings[i-1], axis=1).tolist())
    trainY.append(sequence_encodings[i])
# Inspect trainX and trainY
print("X:\n",'\n'.join([str(item) for item in trainX[0:2]]))
print("Y:\n",'\n'.join([str(item) for item in trainY[0:2]]))

X:
 [[208], [3900]]
[[3900], [1796]]
Y:
 [3900, 1796]
[1796, 1166]


In [219]:
trainX, trainY = [], []
seq_size=2
for i in range(1,len(sequence_encodings)):
    trainX.append(sequence_encodings[i-1])
    trainY.append(sequence_encodings[i])
# Inspect trainX and trainY
print("X:\n",'\n'.join([str(item) for item in trainX[0:2]]))
print("Y:\n",'\n'.join([str(item) for item in trainY[0:2]]))


X:
 [208, 3900]
[3900, 1796]
Y:
 [3900, 1796]
[1796, 1166]


In [223]:
# Build computational graph
input_dim=1 # dim > 1 for multivariate time series
hidden_dim=100 # number of hiddent units h
graph = tf.Graph()
with graph.as_default():
    # input place holders
    # input Shape: [# training examples, sequence length, # features]
    x = tf.placeholder(tf.float32,[None,seq_size,input_dim])
    # label Shape: [# training examples, sequence length]
    y = tf.placeholder(tf.float32,[None,seq_size])
    
    # RNN Network
    cell = rnn.BasicRNNCell(hidden_dim)
    
    # RNN output Shape: [# training examples, sequence length, # hidden] 
    outputs, _ = tf.nn.dynamic_rnn(cell,x,dtype=tf.float32)
    
    
    # weights for output dense layer (i.e., after RNN)
    # W shape: [# hidden, 1]
    W_out = tf.Variable(tf.random_normal([hidden_dim,1]),name="w_out") 
    # b shape: [1]
    b_out = tf.Variable(tf.random_normal([1]),name="b_out")

    # output dense layer:
    num_examples = tf.shape(x)[0] 
    # convert W from [# hidden, 1] to [# training examples, # hidden, 1]
    # step 1: add a new dimension at index 0 using tf.expand_dims
    w_exp= tf.expand_dims(W_out,0)
    # step 2: duplicate W for 'num_examples' times using tf.tile
    W_repeated = tf.tile(w_exp,[num_examples,1,1])
    
    # Dense Layer calculation: 
    # [# training examples, sequence length, # hidden] *
    # [# training examples, # hidden, 1] = [# training examples, sequence length]
    
    y_pred = tf.matmul(outputs,W_repeated)+b_out
    # Actually, y_pred: [# training examples, sequence length, 1]
    # Remove last dimension using tf.squeeze
    y_pred = tf.squeeze(y_pred)
    
    # Cost & Training Step
    cost = tf.reduce_mean(tf.square(y_pred-y))
    train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [224]:
# Run Session
with tf.Session(graph=graph) as sess:
    # initialize variables
    sess.run(tf.global_variables_initializer())
    # Run for 1000 iterations (1000 is arbitrary, need a validation set to tune!)
    print('Training...')
    for i in range(1000): # If we train more, would we overfit? Try 10000
        _, train_err = sess.run([train_op,cost],feed_dict={x:trainX,y:trainY})
        if i==0:
            print('  step, train err= %6d: %8.5f' % (0,train_err)) 
        elif  (i+1) % 100 == 0: 
            print('  step, train err= %6d: %8.5f' % (i+1,train_err)) 

    # Test trained model on training data
    predicted_vals_all= sess.run(y_pred,feed_dict={x:trainX}) 
    # Get last item in each predicted sequence:
    predicted_vals = predicted_vals_all[:,seq_size-1]
    
    # Compute MSE
    # step 1: denormalize data
    predicted_vals_dnorm=predicted_vals*(max_dataset-min_dataset)+min_dataset
    # step 2: get ground-truth
    actual=dataset[seq_size:][:,0]
    # step 3: compute MSE
    mse = ((predicted_vals_dnorm - actual) ** 2).mean()
    print("Training MSE = %10.5f"%mse)
    
    # Plot predictions
    plt.figure()
    plt.plot(list(range(seq_size,seq_size+len(predicted_vals_dnorm))), predicted_vals_dnorm, color='r', label='predicted')
    plt.plot(list(range(len(dataset))), dataset, color='g', label='actual')
    plt.legend()

Training...
  step, train err=      0: 6650120.50000
  step, train err=    100: 5920318.50000
  step, train err=    200: 5519335.50000
  step, train err=    300: 5146309.50000
  step, train err=    400: 4799859.00000
  step, train err=    500: 4478690.00000
  step, train err=    600: 4181554.75000
  step, train err=    700: 3907129.50000
  step, train err=    800: 3654325.50000
  step, train err=    900: 3421941.00000
  step, train err=   1000: 3208951.25000


NameError: name 'max_dataset' is not defined